# Cleaning TCR data (year 2023)

Assumptions:
* Bjf is part of Norra Malmbanan
* Gim is part of Holmsund - Boden Södra inklusive tvärbanor


In the following, we clean up TCR data and process it (e.g., include bandelar, match to contract regions, add length) before exporting it for Power BI visualisation/comparison to cleaned contract data.

## Import data

We first start by reading the cleaned (by OpenRefine) and matched TCRs for 2024 which are matched with contracts number (note that for T23, the matching did not work since the contract numbers are outdated/not-valid!).

In [2]:
import pandas as pd # type: ignore

# Load the CSV file containing matched TCRs for 2024
csv_file_path = "./matched_data_python/TCR_T23_matched.csv"

# Read the CSV file into a DataFrame
tcr_df = pd.read_csv(csv_file_path)

In [3]:
len(tcr_df)

96717

We would also need a BIS dictionary to find missing Bandelnr (and later to construct a graph and find the length of shortest path). We first read the BIS dictionary.

In [4]:
# File and sheet details
excel_file_path = "./raw_data/BIS-data 2024-01-09 - Bandel, plats och förbindelselinje, alla spår.xlsx"

# Load the Excel file
bis_df = pd.read_excel(excel_file_path)

In [5]:
# read the following Trafikplats_jvg_förenklad.csv file
trafikplats_file_path = "./raw_data/Trafikplats_jvg_förenklad.csv"
# Read the CSV file into a DataFrame with ; as separator
trafikplats_df = pd.read_csv(trafikplats_file_path, sep=",")

## Basic clean up

We first do some basic clean, e.g., remove irrelevant columns, set the right datatypes, etc.

In [6]:
# remove columns Relaterade TPÅ:er, and Relaterad åtgärdsnummer
#tcr_df_cleaned = tcr_df.drop(columns=['Relaterade TPÅ:er', 'Relaterad åtgärdsnummer'])
tcr_df_cleaned = tcr_df.copy()

# set columns Starttid and Sluttid to datetime (format 2023-03-15 00:15:00)
tcr_df_cleaned['Starttid'] = pd.to_datetime(tcr_df_cleaned['Starttid'], format='%Y-%m-%d %H:%M:%S')
tcr_df_cleaned['Sluttid'] = pd.to_datetime(tcr_df_cleaned['Sluttid'], format='%Y-%m-%d %H:%M:%S')

# set column Från trafikplats to string
tcr_df_cleaned['Från trafikplats'] = tcr_df_cleaned['Från trafikplats'].astype(str)

We then need to clean up the dictionary by focusing on main tracks (nhsp) and removing duplicates, to speed up the matchning/search.

In [7]:
# for finding the code of the stations
bis_df_no_duplicates = bis_df[['BdlNr', 'Bandel', 'Plats_sign', 'Plats']].drop_duplicates()
# remove all rows with mising values of 'Plats_sign'
bis_df_no_duplicates = bis_df_no_duplicates[bis_df_no_duplicates['Plats_sign'].notna()]

# Focus on the main tracks
# remove all rows where BdlNr is equal to 1 (Ingår ej i bandelsindelning) but not rows where Forbind is not empty (helpful for finding shortest path)
bis_df = bis_df[(bis_df['BdlNr'] != 1) | (bis_df['Forbind'].notna())]

# keep only rows where column Spår_huvud_sido is nhsp
bis_df_nhsp = bis_df[bis_df["Spår_huvud_sido"] == "nhsp"]

# Step 1: Remove duplicates from the mapping
bis_df_nhsp_no_duplicates = bis_df_nhsp[['BdlNr', 'Bandel', 'Plats_sign', 'Plats', 'Forbind']].drop_duplicates()

We also reorganize the same dataframe in order to easily get the length of the track section (Banlangd).

In [8]:
# Step 1: Group by 'BdlNr', 'Bandel', 'Plats_sign', 'Plats' and sum 'Banlangd'
grouped_by_plats = bis_df_nhsp.groupby(['BdlNr', 'Bandel', 'Plats_sign', 'Plats'])['Banlangd'].sum().reset_index()

# Step 2: Group by 'BdlNr', 'Bandel', 'Forbind' and sum 'Banlangd'
grouped_by_forbind = bis_df_nhsp.groupby(['BdlNr', 'Bandel', 'Forbind'])['Banlangd'].sum().reset_index()

# Step 3: Add 'Plats_sign' and 'Plats' columns with NaN to 'grouped_by_forbind' for consistency
grouped_by_forbind['Plats_sign'] = pd.NA
grouped_by_forbind['Plats'] = pd.NA

# Step 4: Add 'Forbind' column with NaN to 'grouped_by_plats' for consistency
grouped_by_plats['Forbind'] = pd.NA

# Step 5: Combine the two DataFrames using outer concatenation
combined_bis_df_nhsp_langd = pd.concat([grouped_by_plats, grouped_by_forbind], ignore_index=True, sort=False)

## Processings

### Add bandelar using dictionary

We first add the column identified_BdlN.

In [9]:
# add column identified_BdlN to tcr_df_cleaned in tcr_df_processed
tcr_df_processed = tcr_df_cleaned.copy()
tcr_df_processed['identified_BdlNr'] = pd.NA

Based on the column Från trafikplats, we can identify the corresponding bandel using the BIS dictionary that we constructed earlier.

Here, we first start with the rows where the driftplats is included (Från inkluderad = Helt).

In [10]:
# Step 1: Create a mapping from 'Plats_sign' to 'BdlNr' for fast lookups
trafikplats_to_bandel_map_bis = bis_df_no_duplicates.set_index('Plats_sign')['BdlNr'].to_dict()

# Step 2: Use .loc for vectorized lookup
mask = tcr_df_processed['identified_BdlNr'].isna() #tcr_df_processed['Från inkluderad'] == 'Helt'
tcr_df_processed.loc[mask, 'identified_BdlNr'] = tcr_df_processed.loc[mask, 'Från trafikplats'].map(trafikplats_to_bandel_map_bis)

In [11]:
# # print the unique values of identified_BdlNr for each value of TCR-id
# print(tcr_df_processed.groupby('TCR-id')['identified_BdlNr'].unique())
# print only the ones with more than two distinct values
print(tcr_df_processed.groupby('TCR-id')['identified_BdlNr'].unique()[tcr_df_processed.groupby('TCR-id')['identified_BdlNr'].unique().apply(lambda x: len(x) >= 2)])

TCR-id
136       [942.0, 952.0, nan]
154     [901.0, 961.0, 962.0]
183     [909.0, 941.0, 942.0]
195            [820.0, 821.0]
198       [942.0, 952.0, nan]
                ...          
1784           [901.0, 902.0]
1786    [925.0, 901.0, 902.0]
1828      [912.0, nan, 901.0]
1834           [711.0, 715.0]
1843    [138.0, 146.0, 147.0]
Name: identified_BdlNr, Length: 366, dtype: object


The previous output shows that there are TCRs with the same id but covering multiple bandelnr. So, we have to be careful not to just directly match TCR with a single bandelnr.

In [12]:
# trafikplats_df to consutruct a mapping from signatur to bandel
trafikplats_df = trafikplats_df.dropna(subset=['signatur'])
# drop rows from trafikplats_df where bandel is equal to 1
trafikplats_df = trafikplats_df[trafikplats_df['bandel'] != 1]

trafikplats_df['signatur'] = trafikplats_df['signatur'].astype(str)
trafikplats_df['bandel'] = trafikplats_df['bandel'].astype(str)
trafikplats_to_bandel_map = trafikplats_df.set_index('signatur')['bandel'].to_dict()

# Ensure the identified_BdlNr column is of type object
tcr_df_processed['identified_BdlNr'] = tcr_df_processed['identified_BdlNr'].astype(object)

# use the mapping to find missing values of identified_BdlNr
mask = tcr_df_processed['identified_BdlNr'].isna()
tcr_df_processed.loc[mask, 'identified_BdlNr'] = tcr_df_processed.loc[mask, 'Från trafikplats'].map(trafikplats_to_bandel_map)

One way to get rid of rows missing identified_BdlNr is to skip/drop the rows that are included (tcr_df_processed['Från inkluderad'] == 'Helt') and are between two rows where  identified_BdlNr is not missing and is the same. These can be found by grouping (same TCR and Starttid) and using their Platssekvensnummer.

In [13]:
# Group by 'TCR-id' and 'Starttid'
to_drop_indices = []

for (tcr_id, starttid), group in tcr_df_processed.groupby(['TCR-id', 'Starttid']):
    # Sort by 'Platssekvensnummer'
    group = group.sort_values(by='Platssekvensnummer')
    group_indices = group.index.tolist()

    # Track a block of consecutive rows with missing 'identified_BdlNr'
    block_start = None

    for i in range(1, len(group) - 1):  # Explicitly exclude first and last rows
        current_row = group.iloc[i]

        # Check if the current row has a missing 'identified_BdlNr'
        if pd.isna(current_row['identified_BdlNr']):
            # Start tracking the block if not already started
            if block_start is None:
                block_start = i
        else:
            # If the block ends, check if it's surrounded by matching 'identified_BdlNr'
            if block_start is not None:
                block_end = i - 1  # End of the block
                # Check if there are surrounding rows with matching 'identified_BdlNr'
                if block_start > 0 and block_end < len(group) - 1:
                    previous_row = group.iloc[block_start - 1]
                    next_row = group.iloc[block_end + 1]
                    if (
                        pd.notna(previous_row['identified_BdlNr']) and
                        pd.notna(next_row['identified_BdlNr']) and
                        previous_row['identified_BdlNr'] == next_row['identified_BdlNr']
                    ):
                        # Add all rows in the block to the drop list
                        to_drop_indices.extend(group_indices[block_start:block_end + 1])
                # Reset the block tracker
                block_start = None

    # Handle a block that ends at the last row (if necessary)
    if block_start is not None and block_start > 0 and block_start < len(group) - 1:
        previous_row = group.iloc[block_start - 1]
        if pd.notna(previous_row['identified_BdlNr']):
            to_drop_indices.extend(group_indices[block_start:])

# Drop the identified rows from the DataFrame
tcr_df_processed = tcr_df_processed.drop(index=to_drop_indices).reset_index(drop=True)

### Handle rows with missing identified_BdlNr

Let us show first some of these rows.

In [14]:
# get the rows where 'Från inkluderad' is 'Helt' and 'identified_BdlNr' is NaN
mask = tcr_df_processed['identified_BdlNr'].isna() & (tcr_df_processed['Från inkluderad'] == 'Helt')
# get the unique values of 'Från trafikplats' where 'Från inkluderad' is 'Helt' and 'identified_BdlNr' is NaN
unique_trafikplats = tcr_df_processed.loc[mask, 'Från trafikplats'].unique()
# print the unique values (sorted in alphabetic order) of 'Från trafikplats' where 'Från inkluderad' is 'Helt' and 'identified_BdlNr' is NaN
print(sorted(unique_trafikplats))

['Bjgr', 'Kogr', 'Mdm', 'Rus']


These four trafikplats are not identified (['Bjgr', 'Kogr', 'Mdm', 'Rus']) and need therefore to be replaced manually with supposedly the closest existing driftplats.

In [15]:
# For rows in tcr_df_processed with Från trafikplats = Bjgr, Replace Bjgr with 'Bjf'
tcr_df_processed.loc[tcr_df_processed['Från trafikplats'] == 'Bjgr', 'Från trafikplats'] = 'Bjf'
# Replace Kogr with Ko
tcr_df_processed.loc[tcr_df_processed['Från trafikplats'] == 'Kogr', 'Från trafikplats'] = 'Ko'
# replace Rus with with Skm (Stenkumla)
tcr_df_processed.loc[tcr_df_processed['Från trafikplats'] == 'Rus', 'Från trafikplats'] = 'Skm'
# remove Mdm
tcr_df_processed = tcr_df_processed[tcr_df_processed['Från trafikplats'] != 'Mdm']
# create a copy of tcr_df_processed
tcr_df_processed_cleaned = tcr_df_processed.copy()

# Use the mapping to update identified_BdlNr after replacing the trafikplats values
# For Bjf
tcr_df_processed_cleaned.loc[tcr_df_processed_cleaned['Från trafikplats'] == 'Bjf', 'identified_BdlNr'] = tcr_df_processed_cleaned.loc[tcr_df_processed_cleaned['Från trafikplats'] == 'Bjf', 'Från trafikplats'].map(trafikplats_to_bandel_map_bis)
# For Ko
tcr_df_processed_cleaned.loc[tcr_df_processed_cleaned['Från trafikplats'] == 'Ko', 'identified_BdlNr'] = tcr_df_processed_cleaned.loc[tcr_df_processed_cleaned['Från trafikplats'] == 'Ko', 'Från trafikplats'].map(trafikplats_to_bandel_map_bis)
# For Skm
tcr_df_processed_cleaned.loc[tcr_df_processed_cleaned['Från trafikplats'] == 'Skm', 'identified_BdlNr'] = tcr_df_processed_cleaned.loc[tcr_df_processed_cleaned['Från trafikplats'] == 'Skm', 'Från trafikplats'].map(trafikplats_to_bandel_map_bis)

In [16]:
# make sure the column identified_BdlNr is of integer number
tcr_df_processed_cleaned['identified_BdlNr'] = tcr_df_processed_cleaned['identified_BdlNr'].astype(int)

### Handle rows not in the graph

Show the rows that cannot be found in the graph. Use the bis_no_duplicates.

In [17]:
# Group by 'TCR-id' and 'Starttid'
to_drop_indices = []

for (tcr_id, starttid), group in tcr_df_processed_cleaned.groupby(['TCR-id', 'Starttid']):
    # Sort the group by 'Platssekvensnummer'
    group = group.sort_values(by='Platssekvensnummer')

    # Exclude the first and last rows
    middle_rows = group.iloc[1:-1]  # Exclude first and last rows

    # Find rows where 'Från trafikplats' is not in the mapping
    missing_in_mapping = middle_rows.loc[
        ~middle_rows['Från trafikplats'].isin(trafikplats_to_bandel_map_bis.keys())
    ]

    # Print and collect indices of rows to drop
    for index, row in missing_in_mapping.iterrows():
        print(f"Dropping row with 'Från trafikplats': {row['Från trafikplats']} (TCR-id: {tcr_id}, Starttid: {starttid})")
        to_drop_indices.append(index)

# Drop the identified rows from the DataFrame
tcr_df_processed_cleaned = tcr_df_processed_cleaned.drop(index=to_drop_indices).reset_index(drop=True)

Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-02-27 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-02-28 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-01 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-02 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-06 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-07 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-08 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-09 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-13 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-14 22:00:00)
Dropping row with 'Från trafikplats': Stlv (TCR-id: 200, Starttid: 2023-03-15 22:00:00)
Dropping row with 'Från trafikpl

In [18]:
# Initialize a set to store unique 'Från trafikplats' values that are missing
missing_trafikplats_set = set()

# Group by 'TCR-id' and 'Starttid'
for (tcr_id, starttid), group in tcr_df_processed_cleaned.groupby(['TCR-id', 'Starttid']):
    # Sort the group by 'Platssekvensnummer'
    group = group.sort_values(by='Platssekvensnummer')

    # Check if any 'Från trafikplats' in middle rows is not in the mapping
    missing_in_mapping = group.loc[
        ~group['Från trafikplats'].isin(trafikplats_to_bandel_map_bis.keys())
    ]

    # Add the missing 'Från trafikplats' values to the set
    missing_trafikplats_set.update(missing_in_mapping['Från trafikplats'].unique())

# Output the unique missing 'Från trafikplats' values
if missing_trafikplats_set:
    print("Unique 'Från trafikplats' values missing from the mapping:")
    print(missing_trafikplats_set)
else:
    print("All 'Från trafikplats' values are present in the mapping.")


Unique 'Från trafikplats' values missing from the mapping:
{'Les', 'Sta', 'Ksc', 'Tul'}


### Reformat to add förbindelser

We can now remove rows in between so that we only keep the first and last row

In [19]:
def get_first_last_rows(group):
    # Get first and last rows as Series
    first_row = group.iloc[0]
    last_row = group.iloc[-1]
    
    # Convert Series to DataFrame before concatenation
    first_df = pd.DataFrame(first_row).T
    last_df = pd.DataFrame(last_row).T

    # Ensure TCR-id, Starttid, and identified_BdlNr from first row are preserved
    last_df['TCR-id'] = first_row['TCR-id']
    last_df['Starttid'] = first_row['Starttid']
    last_df['identified_BdlNr'] = first_row['identified_BdlNr']
    
    # Concatenate vertically
    result = pd.concat([first_df, last_df], axis=0)
    
    # Reset index to avoid duplicates
    result = result.reset_index(drop=True)
    
    return result

# Apply the function
filtered_tcr_df = tcr_df_processed_cleaned.groupby(
    ['TCR-id', 'Starttid', 'identified_BdlNr'], 
    as_index=False, 
    group_keys=False
).apply(get_first_last_rows)

# Reset index if needed
filtered_tcr_df = filtered_tcr_df.reset_index(drop=True)

C:\Users\AbdouAA\AppData\Local\Temp\ipykernel_21232\2367047098.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_tcr_df = tcr_df_processed_cleaned.groupby(


Before filling in missing bandelar (and calculating the length between two consecutive places/rows), we need to reformat the tcr_df so that in each row we combine the row with the next row (in Platssekvensnummer), if any (until final row in the sequence).

In [20]:
# Step 1: Sort the DataFrame by 'TCR-id' and 'Platssekvensnummer'
tcr_df_reformat = filtered_tcr_df.copy()
tcr_df_reformat = tcr_df_reformat.sort_values(by=['TCR-id', 'Starttid', 'Platssekvensnummer']).reset_index(drop=True)

# Step 4: Create 'next_trafikplats' and 'next_Från_inkluderad'
tcr_df_reformat['next_trafikplats'] = tcr_df_reformat.groupby(['TCR-id', 'Starttid'])['Från trafikplats'].shift(-1)

# set missing value
tcr_df_reformat['next_Från_inkluderad'] = pd.NA

# also add the next_identified_BdlNr
tcr_df_reformat['next_identified_BdlNr'] = tcr_df_reformat.groupby(['TCR-id', 'Starttid'])['identified_BdlNr'].shift(-1)

We create a new column called förbind_list which will contactenate Från trafikplats of two consecutive rows, e.g., A-B (where A is trafikplats of the first row and B is the second), next row will have B-C, etc. until the final förbind in the sequence.

In [21]:
# Step 5: Create 'förbind_list' with conditional parentheses
def format_trafikplats(trafikplats, inkluderad):
    """Format trafikplats name with parentheses based on inclusion status."""
    # check if inkluderad is not missing
    if pd.isna(inkluderad) or inkluderad != 'Helt':
        return f"({trafikplats})"
    return trafikplats

def create_förbind(row):
    """Create förbind string for a row, connecting two trafikplats names."""
    if pd.isna(row['next_trafikplats']):
        if row['Från inkluderad'] != 'Helt':
            return None
        else:
            return f"{row['Från trafikplats']}"
        

    if(row['Från trafikplats'] == row['next_trafikplats']):
        if row['Från inkluderad'] == 'Helt':
            return f"{row['Från trafikplats']}"
        else:
            return None

    from_tp = format_trafikplats(row['Från trafikplats'], row['Från inkluderad'])
    to_tp = format_trafikplats(row['next_trafikplats'], row['next_Från_inkluderad'])

    # return both förbind and identified_BdlNr
    return f"{from_tp}-{to_tp}"

# add a column where you put 
# Apply the function to create förbind_list
tcr_df_reformat['förbind_list'] = tcr_df_reformat.apply(create_förbind, axis=1)

Last, we should remove rows with no förbind_list and do some clean (remove unwanted columns).

In [22]:
# Step 6: Remove temporary 'next_trafikplats' and 'next_Från_inkluderad' columns
#tcr_df_langd = tcr_df_reformat.drop(columns=['next_trafikplats', 'next_Från_inkluderad'])

# Step 7: Remove the final row in each sequence
tcr_df_langd = tcr_df_reformat.dropna(subset=['förbind_list']).reset_index(drop=True)

## Calculate Banlängd

There are some rows where förbind_list is a single stations.

In [23]:
# add a new column named 'Banlangd' to tcr_df_langd
tcr_df_langd['Banlangd'] = pd.NA

# iterate over rows in servicekontrakt_df_langd (with missing values of langd and Bandelnamn without "-")
# and find the corresponding Banlangd using short_path = combined_bis_df_nhsp_langd['Plats']
for index, row in tcr_df_langd.iterrows():
    # if Bandelnamn has "-" just continue
    if "-" in row['förbind_list']:
        continue
    # if not find the corresponding Banlangd using short_path
    bandel_nr = row['identified_BdlNr']
    driftplats_sign = row['förbind_list']
    # find the corresponding Banlangd in combined_bis_df_nhsp_langd
    banlangd = combined_bis_df_nhsp_langd[(combined_bis_df_nhsp_langd['BdlNr'] == bandel_nr) & (combined_bis_df_nhsp_langd['Plats_sign'] == driftplats_sign)]['Banlangd']
    if len(banlangd) > 0:
        # sum up the Banlangd values
        tcr_df_langd.at[index, 'Banlangd'] = banlangd.sum()
    else:
        # put zero if no Banlangd found
        tcr_df_langd.at[index, 'Banlangd'] = 0

### Graph construction

To find the length for the rows, we will need to construct a graph of the network and find the shortest path and then accumulate the length on the main tracks if any.

In [24]:
# Step 1: Group by 'BdlNr', 'Bandel', 'Plats_sign', 'Plats' and sum 'Banlangd' where 'Spår_huvud_sido' is 'nhsp'
grouped_by_plats_nhsp = bis_df[bis_df['Spår_huvud_sido'] == 'nhsp'].groupby(['BdlNr', 'Bandel', 'Plats_sign', 'Plats'])['Banlangd'].sum().reset_index()

# Step 2: Group by 'BdlNr', 'Bandel', 'Forbind' and sum 'Banlangd' where 'Spår_huvud_sido' is 'nhsp'
grouped_by_forbind_nhsp = bis_df[bis_df['Spår_huvud_sido'] == 'nhsp'].groupby(['BdlNr', 'Bandel', 'Forbind'])['Banlangd'].sum().reset_index()

# Step 3: Group by 'BdlNr', 'Bandel', 'Plats_sign', 'Plats' where 'Spår_huvud_sido' is not 'nhsp' and set 'Banlangd' to 0
grouped_by_plats_non_nhsp = bis_df[bis_df['Spår_huvud_sido'] != 'nhsp'].groupby(['BdlNr', 'Bandel', 'Plats_sign', 'Plats']).size().reset_index(name='Banlangd')
grouped_by_plats_non_nhsp['Banlangd'] = 0

# Step 4: Group by 'BdlNr', 'Bandel', 'Forbind' where 'Spår_huvud_sido' is not 'nhsp' and set 'Banlangd' to 0
grouped_by_forbind_non_nhsp = bis_df[bis_df['Spår_huvud_sido'] != 'nhsp'].groupby(['BdlNr', 'Bandel', 'Forbind']).size().reset_index(name='Banlangd')
grouped_by_forbind_non_nhsp['Banlangd'] = 0

# Combine nhsp and non-nhsp dataframes
grouped_by_plats = pd.concat([grouped_by_plats_nhsp, grouped_by_plats_non_nhsp], ignore_index=True)
grouped_by_forbind = pd.concat([grouped_by_forbind_nhsp, grouped_by_forbind_non_nhsp], ignore_index=True)

# when when duplicate rows are present, keep the none with the highest Banlangd
grouped_by_plats = grouped_by_plats.sort_values('Banlangd', ascending=False).drop_duplicates(['BdlNr', 'Bandel', 'Plats_sign', 'Plats']).sort_index()
grouped_by_forbind = grouped_by_forbind.sort_values('Banlangd', ascending=False).drop_duplicates(['BdlNr', 'Bandel', 'Forbind']).sort_index()

# Step 3: Add 'Plats_sign' and 'Plats' columns with NaN to 'grouped_by_forbind' for consistency
grouped_by_forbind['Plats_sign'] = pd.NA
grouped_by_forbind['Plats'] = pd.NA

# Step 4: Add 'Forbind' column with NaN to 'grouped_by_plats' for consistency
grouped_by_plats['Forbind'] = pd.NA

# Step 5: Combine the two DataFrames using outer concatenation
combined_bis_df_langd = pd.concat([grouped_by_plats, grouped_by_forbind], ignore_index=True, sort=False)

In [25]:
import networkx as nx
import pandas as pd

# Global cache for lengths and the graph
langd_cache = {}
GLOBAL_GRAPH = None

# Step 1: Create a mapping from Plats_sign (full name) to Banlangd
station_length_lookup = combined_bis_df_langd.set_index('Plats_sign')['Banlangd'].to_dict()

### Utility Functions ###

def initialize_global_graph(dictionary_df):
    """Initialize the global graph once"""
    global GLOBAL_GRAPH
    if GLOBAL_GRAPH is None:
        #bdl_df = dictionary_df[(dictionary_df['BdlNr'] >= 2) & (dictionary_df['BdlNr'] <= 990)]
        bdl_df = dictionary_df
        GLOBAL_GRAPH = nx.Graph()  # Undirected graph to simulate bidirectional connections
        for _, row in bdl_df.iterrows():
            if pd.notna(row['Forbind']):
                start, end = row['Forbind'].split('-')
                length = row['Banlangd']
                GLOBAL_GRAPH.add_edge(start.strip(), end.strip(), length=length)

def calculate_sum_langd(forbind_list, dictionary_df):
    if not forbind_list or forbind_list == '':
        # print that the forbind_list is empty
        print("forbind_list is empty")
        return None
    
    # Check cache
    cache_key = (forbind_list)
    if cache_key in langd_cache:
        return langd_cache[cache_key]
    
    # Initialize global graph if not already done
    if GLOBAL_GRAPH is None:
        initialize_global_graph(dictionary_df)
    
    # Split and clean the forbind_list
    forbinds = [f.strip() for f in forbind_list.split(',')]
    stations = [station for forbind in forbinds for station in forbind.split('-')]
    first_station = stations[0]
    last_station = stations[-1]
    
    # Check if first and last stations are enclosed in parentheses
    include_first_station = not (first_station.startswith('(') and first_station.endswith(')'))
    include_last_station = not (last_station.startswith('(') and last_station.endswith(')'))
    
    # Remove parentheses for lookup in the graph
    first_station_cleaned = first_station.strip('()')
    last_station_cleaned = last_station.strip('()')
    
    # Check if stations exist in graph
    if first_station_cleaned not in GLOBAL_GRAPH or last_station_cleaned not in GLOBAL_GRAPH:
        langd_cache[cache_key] = None
        # print the names of the stations not found in the graph
        if first_station_cleaned not in GLOBAL_GRAPH:
            print(f"First station {first_station} not found in graph, part of {forbind_list}")
        if last_station_cleaned not in GLOBAL_GRAPH:
            print(f"Last station {last_station} not found in graph, part of {forbind_list}")
        return None
    
    try:
        path_length = nx.shortest_path_length(
            GLOBAL_GRAPH, 
            source=first_station_cleaned, 
            target=last_station_cleaned, 
            weight='length'
        )
        
        # Calculate length of intermediate stations
        shortest_path_stations = nx.shortest_path(
            GLOBAL_GRAPH, 
            source=first_station_cleaned, 
            target=last_station_cleaned
        )
        intermediate_stations = shortest_path_stations[1:-1]  # Exclude first and last station
        station_length_sum = sum(station_length_lookup.get(station, 0) for station in intermediate_stations)
        
        # Add lengths of first and last stations based on inclusion rules
        if include_first_station:
            station_length_sum += station_length_lookup.get(first_station_cleaned, 0)
        if include_last_station:
            station_length_sum += station_length_lookup.get(last_station_cleaned, 0)

        total_length = path_length + station_length_sum
        langd_cache[cache_key] = total_length
        return total_length
        
    except nx.NetworkXNoPath:
        langd_cache[cache_key] = None
        return None  # No path found

### Get langd (from shortest path)

In [26]:
def calculate_sum_langd_for_bandelnamn(row, dictionary_df):

    row_bandel = None
    row_forbind = None
    if pd.notna(row['identified_BdlNr']): # for TCRs
        row_bandel = int(row['identified_BdlNr'])
    row_forbind = row['förbind_list']

    # Case 1: Single station
    if '-' not in row_forbind:
        single_station = row_forbind
        
        # Find the row in dictionary_df where Plats_sign matches the single station
        matching_station = dictionary_df[dictionary_df['Plats_sign'] == single_station]
        
    
        # keep only rows where BdlNr is same as row['Bandelnr']
        matching_station = matching_station[matching_station['BdlNr'] == row_bandel]

        # If no matching station is found, return None
        if matching_station.empty:
            print(f"No matching station found for {single_station}")
            return None
        
        # Get the station's length as the sum of all values in matching_station['Banlangd']
        # station_length = matching_station['Banlangd'].iloc[0]
        station_length = matching_station['Banlangd'].sum()
        
        return station_length

    # Case 2: Multiple stations (existing logic)
    return calculate_sum_langd(row_forbind, dictionary_df)


tcr_df_langd.loc[tcr_df_langd['Banlangd'].isna(), 'Banlangd'] = tcr_df_langd[tcr_df_langd['Banlangd'].isna()].apply(
    lambda row: calculate_sum_langd_for_bandelnamn(row, combined_bis_df_langd),
    axis=1
)

# rename the column 'Banlangd' to 'sum_langd'
tcr_df_langd = tcr_df_langd.rename(columns={'Banlangd': 'sum_langd'})

# make sure column sum_langd is a real number
tcr_df_langd['sum_langd'] = pd.to_numeric(tcr_df_langd['sum_langd'], errors='coerce')

First station Tul not found in graph, part of Tul-(Söd)
Last station (Sta) not found in graph, part of Hu-(Sta)
Last station (Sta) not found in graph, part of (Åbe)-(Sta)
Last station (Les) not found in graph, part of Tbn-(Les)
Last station (Ksc) not found in graph, part of Sån-(Ksc)
First station Ksc not found in graph, part of Ksc-(Skog)
First station Ksc not found in graph, part of Ksc-(Kvä)


Since there are some stations could not be found in the graph {'Ksc', 'Sta', 'Tul', 'Les'}, we do the following manual adjustements:
* Replace "Tul" with closest "Uts"
* replace Ksc with Ks
* replace Les with Alh
* For (Sta)
    * Replace "(Åbe)-(Sta)" with closest "(Åbe)-(Hu)"
    * Replace "Hu-(Sta)" with "Hu-(Äs)"

In [27]:
# in tcr_df_langd, replace in column förbind_list all occurences of 'Tul' with 'Uts'
tcr_df_langd.loc[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0), 'förbind_list'] = tcr_df_langd.loc[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0), 'förbind_list'].str.replace('Tul', 'Uts')


In [28]:
# same for "Ksc" with "Ks"
tcr_df_langd.loc[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0), 'förbind_list'] = tcr_df_langd[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0)]['förbind_list'].str.replace('Ksc', 'Ks')
# same for Les with Alh
tcr_df_langd.loc[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0), 'förbind_list'] = tcr_df_langd[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0)]['förbind_list'].str.replace('Les', 'Alh')
# same for (Åbe)-(Sta) with (Åbe)-(Hud)
tcr_df_langd.loc[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0), 'förbind_list'] = tcr_df_langd[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0)]['förbind_list'].str.replace('(Åbe)-(Sta)', '(Åbe)-(Hu)')
# same for Hu-(Sta) with Hu-(Äs)
tcr_df_langd.loc[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0), 'förbind_list'] = tcr_df_langd[(tcr_df_langd['sum_langd'].isna()) | (tcr_df_langd['sum_langd'] == 0)]['förbind_list'].str.replace('Hu-(Sta)', 'Hu-(Äs)')

# do not forget to update the identified_BdlNr column for these rows
# For Uts
#tcr_df_langd.loc[tcr_df_langd['förbind_list'].str.contains('Uts'), 'identified_BdlNr'] = tcr_df_langd.loc[tcr_df_langd['förbind_list'].str.contains('Uts'), 'förbind_list'].map(trafikplats_to_bandel_map_bis)
# For Hu
#tcr_df_langd.loc[tcr_df_langd['förbind_list'].str.contains('Hu-(Äs)'), 'next_identified_BdlNr'] = tcr_df_langd.loc[tcr_df_langd['förbind_list'].str.contains('Hu-(Äs)'), 'förbind_list'].map(trafikplats_to_bandel_map_bis)


In [29]:
# drop rows where förbind_list is equal to Sat
tcr_df_langd = tcr_df_langd[tcr_df_langd['förbind_list'] != 'Sta']

We can now run the shortest path again on these rows.

In [30]:
tcr_df_langd.loc[tcr_df_langd['sum_langd'].isna(), 'sum_langd'] = tcr_df_langd[tcr_df_langd['sum_langd'].isna()].apply(
    lambda row: calculate_sum_langd_for_bandelnamn(row, combined_bis_df_langd),
    axis=1
)

## Matching to contracts

### Reading BIS-contract file

Load the excel file containing BIS information for mapping the bandel number with contract name. The file name is BIS_24_kontrakt_bandel_plats.xlsx and has sheet BIS 2024-01-09 with columns such as Bandel_nummer, UH_kontraktsområde.

In [31]:
# File and sheet details
excel_file_path = "./raw_data/BIS_24_kontrakt_bandel_plats.xlsx"
sheet_name = "BIS 2024-01-09"

# Load the Excel file
bis_kontrakt_df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

In [32]:
# Step 1: Remove duplicates from the mapping
bandel_plats_to_contract_map = bis_kontrakt_df[['Bandel_nummer', 'Plats_sign','UH_kontraktsområde']].drop_duplicates()

# Step 2: Filter out rows where UH_kontraktsområde is NaN or 'Ingår inte i något kontrakt'
bandel_plats_to_contract_map = bandel_plats_to_contract_map[
    bandel_plats_to_contract_map['UH_kontraktsområde'].notna() & 
    (bandel_plats_to_contract_map['UH_kontraktsområde'] != 'Ingår inte i något kontrakt')
]

### Map to contracts

Initially we create a new column and fill it progressively with matchings. We start with the case where both Bandelnr and single station is present.

In [33]:
# create an empty column kontrakt_från_bandel in tcr_df_langd_kontrakt
tcr_df_langd_kontrakt = tcr_df_langd.copy()
tcr_df_langd_kontrakt['kontrakt_från_bandel'] = pd.NA

# for rows with non-empty Bandelnr, and short_path not containing "-" find the corresponding kontrakt_från_bandel
for index, row in tcr_df_langd_kontrakt.iterrows():
    if '-' not in row['förbind_list']:
        driftplats_sign = row['förbind_list']
        # remove parentheses
        driftplats_sign = driftplats_sign.strip('()')
        # find the corresponding kontrakt_från_bandel in bandel_plats_to_contract_map
        #kontrakt = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Bandel_nummer'] == bandel_nr) & (bandel_plats_to_contract_map['Plats_sign'] == driftplats_sign)]['UH_kontraktsområde']
        kontrakt = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Plats_sign'] == driftplats_sign)]['UH_kontraktsområde']
        if len(kontrakt) == 1:
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt.values[0]
        elif len(kontrakt) > 1:
            bandel_nr = int(row['identified_BdlNr'])
            kontrakt = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Bandel_nummer'] == bandel_nr) & (bandel_plats_to_contract_map['Plats_sign'] == driftplats_sign)]['UH_kontraktsområde']
            if len(kontrakt) == 1:
                tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt.values[0]
                #print("Multiple matches (unique after bandelnr) for : ", row['förbind_list'])
            else:
                # save the first and print that there are multiple matches
                tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt.values[0]
                print("Multiple matches (even after bandelnr) for : ", row['förbind_list'])
        else:
            # put zero if no kontrakt_från_bandel found
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = 'Inget kontrakt'
            print("No match for : ", row['förbind_list'])

We move on to map rows where we have two stations and exactly one of them does not have parentheses.

In [34]:
# now we will map rows with missing kontrakt_från_bandel that have Bandelnamn with "-" in it and no two occurrence of "("  in förbind_list
# we split the förbind_list by "-" and then take element wihout parentheses and find the corresponding kontrakt_från_bandel
for index, row in tcr_df_langd_kontrakt.iterrows():
    if '-' in row['förbind_list'] and row['förbind_list'].count('(') == 1:
        # split the förbind_list by "-"
        bandel_nr = None
        left = None
        stations = row['förbind_list'].split('-')
        if stations[0].startswith('('):
            station_sign = stations[1]
            left = False
            kontrakt = bandel_plats_to_contract_map[bandel_plats_to_contract_map['Plats_sign'] == station_sign]['UH_kontraktsområde'].unique()
        else:
            station_sign = stations[0]
            left = True
            kontrakt = bandel_plats_to_contract_map[bandel_plats_to_contract_map['Plats_sign'] == station_sign]['UH_kontraktsområde'].unique()
        
        if len(kontrakt) == 1:
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt[0]
        elif len(kontrakt) > 1: 
            if left:
                bandel_nr = int(row['identified_BdlNr'])
            else:
                bandel_nr = int(row['next_identified_BdlNr'])
            kontrakt = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Bandel_nummer'] == bandel_nr) & (bandel_plats_to_contract_map['Plats_sign'] == station_sign)]['UH_kontraktsområde'].unique()    
            # save the first and print that there are multiple matches
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt[0]
            if len(kontrakt) == 1:
                tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt[0]
                #print("Multiple matches (but now unique after bandelnr) for : ", row['förbind_list'])
            elif len(kontrakt) > 1:
                # save the first and print that there are multiple matches
                tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt[0]
                print("Multiple matches (even after bandelnr) for : ", row['förbind_list'])
        else:
            # put zero if no kontrakt_från_bandel found
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = 'Inget kontrakt'
            print("No match for : ", row['förbind_list'])

Next, are the rows where no or both parentheses exist in the track section (i.e., both/none ends are included).

In [35]:
for index, row in tcr_df_langd_kontrakt.iterrows():
    if '-' in row['förbind_list'] and ( row['förbind_list'].count('(') == 0 or row['förbind_list'].count('(') == 2):
        # split the förbind_list by "-"
        stations = row['förbind_list'].split('-')
        stations = [station.strip('()') for station in stations]
        kontrakt_1 = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Plats_sign'] == stations[0])]['UH_kontraktsområde'].unique()
        kontrakt_2 = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Plats_sign'] == stations[1])]['UH_kontraktsområde'].unique()
        bandel_nr_1 = int(row['identified_BdlNr'])
        bandel_nr_2 = int(row['next_identified_BdlNr'])
        if len(kontrakt_1) > 1:
            kontrakt_1 = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Bandel_nummer'] == bandel_nr_1) & (bandel_plats_to_contract_map['Plats_sign'] == stations[0])]['UH_kontraktsområde'].unique()
        if len(kontrakt_2) > 1:
            kontrakt_2 = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Bandel_nummer'] == bandel_nr_2) & (bandel_plats_to_contract_map['Plats_sign'] == stations[1])]['UH_kontraktsområde'].unique()
        # kontrakt will be the union of kontrakt_1 and kontrakt_2
        kontrakt = list(set(kontrakt_1) | set(kontrakt_2))
        # keep only unique values
        if len(kontrakt) == 1:
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt[0]
        elif len(kontrakt) > 1:
            if bandel_nr_1 == bandel_nr_2:
                kontrakt_bdlnr = bandel_plats_to_contract_map[(bandel_plats_to_contract_map['Bandel_nummer'] == bandel_nr_1)]['UH_kontraktsområde'].unique()
                if len(kontrakt_bdlnr) == 1:
                    tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = kontrakt_bdlnr[0]
                    continue
            # of the elements in kontrakt find the closest match using fuzzy matching to row['Kontraktsområdesnamn']   
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = 'Flera kontraktområden'
            # print the value of the closest match as well as row['Kontraktsområdesnamn']
            #print(f"Multiple matches for Bandelnamn: {row['förbind_list']} with Kontraktsområdesnamn: {kontrakt}.")
        else:
            # put zero if no kontrakt_från_bandel found
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = 'Inget kontrakt'
            print("No match for Bandelnamn: ", row['förbind_list'])

There are some track sections where the two ends belong to two different contracts. In this case, we choose the contract of the same TCR.

In [36]:
# For rows with kontrakt_från_bandel equal to 'Flera kontraktområden', replace the value with the value from kontrakt_från_bandel of another row with the same TCR-id and Starttid

for index, row in tcr_df_langd_kontrakt.iterrows():
    if row['kontrakt_från_bandel'] == 'Flera kontraktområden':
        # Find rows with the same TCR-id and Starttid
        matching_rows = tcr_df_langd_kontrakt[
            (tcr_df_langd_kontrakt['TCR-id'] == row['TCR-id']) &
            (tcr_df_langd_kontrakt['Starttid'] == row['Starttid']) &
            (tcr_df_langd_kontrakt['kontrakt_från_bandel'] != 'Flera kontraktområden') &
            (tcr_df_langd_kontrakt['kontrakt_från_bandel'] != 'Inget kontrakt') &
            (tcr_df_langd_kontrakt['kontrakt_från_bandel'].notna())
        ]

        # If there are matching rows, pick the first available 'kontrakt_från_bandel'
        if not matching_rows.empty:
            replacement_value = matching_rows.iloc[0]['kontrakt_från_bandel']
            tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = replacement_value
            #print(f"Replaced 'Flera kontraktområden' for TCR-id {row['TCR-id']}, Starttid {row['Starttid']} with '{replacement_value}'.")
        else:
            # Print a warning if no replacement is found
            print(f"No replacement found for TCR-id {row['TCR-id']}, Starttid {row['Starttid']} with 'Flera kontraktområden'.")


No replacement found for TCR-id 1070, Starttid 2023-01-23 10:00:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-01-24 10:00:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-01-25 10:00:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-01-26 10:00:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-01-27 10:00:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-01-30 10:20:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-01-31 10:20:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-02-01 10:20:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-02-02 10:20:00 with 'Flera kontraktområden'.
No replacement found for TCR-id 1070, Starttid 2023-02-03 10:20:00 with 'Flera kontraktområden'.
No replacement found for TCR-i

In [37]:
# check if there are any rows with kontrakt_från_bandel equal to 'Flera kontraktområden' or 'Inget kontrakt', print the number of such rows
print(f"Number of rows with 'Flera kontraktområden': {tcr_df_langd_kontrakt[tcr_df_langd_kontrakt['kontrakt_från_bandel'] == 'Flera kontraktområden'].shape[0]}")
# print unique values of förbind_list for rows with kontrakt_från_bandel equal to 'Flera kontraktområden'
print("Unique values of förbind_list for rows with 'Flera kontraktområden':")
print(tcr_df_langd_kontrakt[tcr_df_langd_kontrakt['kontrakt_från_bandel'] == 'Flera kontraktområden']['förbind_list'].unique())

Number of rows with 'Flera kontraktområden': 330
Unique values of förbind_list for rows with 'Flera kontraktområden':
['(Ttu)-(Bål)' '(Gn)-(Fle)']


There some track sections which belong to bandelar sharing two contract regions. We fix this manually.
* For förbind_list = (Ttu)-(Bål), kontrakt_från_bandel is Mälarbanan
* For förbind_list = (Gn)-(Fle), kontrakt_från_bandel is Västra Södra Stambanan

In [38]:
# Manual fixes for specific 'förbind_list' values
manual_fixes = {
    "(Ttu)-(Bål)": "Mälarbanan",
    "(Gn)-(Fle)": "Västra Södra Stambanan"
}

# Filter rows where 'kontrakt_från_bandel' is 'Flera kontraktområden'
filtered_df_flera_kontrakt = tcr_df_langd_kontrakt[tcr_df_langd_kontrakt['kontrakt_från_bandel'] == 'Flera kontraktområden']

# Apply manual fixes only to the filtered rows
for index, row in filtered_df_flera_kontrakt.iterrows():
    if row['förbind_list'] in manual_fixes:
        tcr_df_langd_kontrakt.at[index, 'kontrakt_från_bandel'] = manual_fixes[row['förbind_list']]
        #print(f"Manually updated 'kontrakt_från_bandel' for {row['förbind_list']} to {manual_fixes[row['förbind_list']]}.")


## Export to Excel files

In [39]:
# add a new column named 'Kontraktsområdesnamn' to tcr_df_langd_kontrakt
tcr_df_langd_kontrakt['Kontraktsområdesnamn'] = pd.NA

# Step 9: Keep only the specified columns in tcr_df
tcr_df_to_export = tcr_df_langd_kontrakt[['TCR-id', 'Klassificering',
                 'Starttid', 'Sluttid', 'Servicefönster_nya_kategorier',
                 'Relaterade TPÅ:er', 'tid_timmar', 'Relaterad åtgärdsnummer',
                 'förbind_list', 'identified_BdlNr', 'sum_langd', 'Kontraktsområdesnamn', 'kontrakt_från_bandel']]

# Step 10: Export the DataFrame to an Excel file
excel_file_path = "./result_data_PowerBI/TCR_T23_matched_bandelar.xlsx"
tcr_df_to_export.to_excel(excel_file_path, index=False)